In [676]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees

import numpy as np
import pandas as pd
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import  matplotlib.pyplot as plt
import pickle

In [742]:
def get_missing(df):
    missing = df.isna().sum()
    missing /= df.shape[0]
    missing *=100
    missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
    return missing


In [743]:
filepath = "datasets/Institutional Finances/F_F2_1415-1920_data.csv"
institutional_finance = pd.read_csv(filepath, index_col=0, low_memory= False)
missing = institutional_finance.isna().sum()
missing /= institutional_finance.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
threshold = 30
institutional_finance_over_threshold_missing = institutional_finance[(missing[missing["Percent Of Missing Values"] < threshold]).index]
missing = institutional_finance_over_threshold_missing.isna().sum()
missing /= institutional_finance_over_threshold_missing.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
# All missing entries with percent missing > 7 are verifiably 0 imputable
missing_keys = (missing[missing["Percent Of Missing Values"] > 7]).index
imputation_dict = {k: 0 for k in missing_keys}

cleaned_institutional_finance = institutional_finance_over_threshold_missing.fillna(imputation_dict)

# Remaining missing values are mean imputed -- very small fraction of rows in any case
num_cols = cleaned_institutional_finance.columns[cleaned_institutional_finance.dtypes.values != 'object']
cleaned_institutional_finance = institutional_finance_over_threshold_missing.fillna(value=cleaned_institutional_finance[num_cols].mean())
cleaned_institutional_finance = cleaned_institutional_finance[num_cols]
cleaned_institutional_finance.drop(columns= "f2d17", inplace = True)
sigma_threshold = 2
cleaned_institutional_finance_no_outliers = cleaned_institutional_finance[(np.abs(stats.zscore(cleaned_institutional_finance)) < sigma_threshold).all(axis=1)]

fnce_data_unique = cleaned_institutional_finance_no_outliers.groupby(['unitid']).mean().reset_index()
fnce_data_final = fnce_data_unique.drop(['year'], axis = 1)


In [744]:
def gen_expense():
    train_features = fnce_data_final.drop(['unitid'], axis = 1)
    model = PCA(n_components=train_features.shape[1]).fit(train_features)
    X_pc = model.transform(train_features)

    # number of components
    n_pcs= model.components_.shape[0]

    # get the index of the most important feature on EACH component
    # LIST COMPREHENSION HERE
    most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

    initial_feature_names = train_features.columns
    # get the names
    most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

    # LIST COMPREHENSION HERE AGAIN
    dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}
    expenses = ['unitid']
    seen_categories = ['13']
    for i in range(n_pcs):
        most_important_name = most_important_names[i]
        category = most_important_name[3:5]

        if 'e' in most_important_name and most_important_name != 'unitid' and most_important_name not in expenses and category not in seen_categories:
            # print(f"name: {most_important_name}, cat: {category}")
            expenses.append(most_important_name)
            seen_categories.append(category)
    # expenses.append('unitid')
    expenses = np.array(expenses)



In [745]:
expenses = ['unitid', 'f2b02', 'f2c05', 'f2c06', 'f2d11', 'f2d12', 'f2e011', 'f2e021', 'f2e031', 'f2e041', 'f2e051', 'f2e061', 'f2e071', 'f2e081']

In [746]:
total_students = pd.read_csv("datasets/12-Month Enrollment/EFFY_2015-2021_data.csv")
total_students = total_students.groupby('unitid').mean().reset_index()
total_students = total_students[['unitid', 'efytotlt']]

In [747]:
fnce_data_final_features = fnce_data_final[expenses]
for c in fnce_data_final_features.columns:
    if c != 'unitid':
        fnce_data_final_features[c] /= fnce_data_final['f2e131']
for c in fnce_data_final_features.columns:
    if c!= 'unitid' and 'e' in c:
        fnce_data_final_features = fnce_data_final_features[fnce_data_final_features[c] < 1]
fnce_data_final_features = pd.merge(fnce_data_final_features, total_students, on='unitid', how='inner')
fnce_data_final_features['expense_ps'] = fnce_data_final_features['f2b02'] / fnce_data_final_features['efytotlt']
fnce_data_final_features = fnce_data_final_features.drop(['efytotlt'], axis = 1)

In [748]:
def test(X, y):
    # if np.count_nonzero(y == 0) > 0:
    #     lr = LinearRegression().fit(X, y)
    #     print(lr.score(X,y))
    # else:
    offset = int(X.shape[0] * 0.85)
    X_train, y_train = X[:offset], y[:offset]
    X_test, y_test = X[offset:], y[offset:]
    reg = LazyRegressor(verbose=1, ignore_warnings=True, custom_metric=None)
    models, predictions = reg.fit(X_train, X_test, y_train, y_test)
    print(models)


In [756]:
def xgb_test(X, y, input_frame):

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)
    grid = {
        'n_estimators': [3, 5, 7, 10, 15, 20, 30, 40],
        'max_depth' : [5, 7, 10, 12, 15, 18],
        'eta' : [0.05, 0.1, 0.12, 0.15, 0.17, 0.2, 0.22, 0.25],
        'grow_policy' : ['depthwise', 'lossguide']
    }

    model_s = XGBRegressor()
    xgbr_cv_s = RandomizedSearchCV(estimator=model_s, param_distributions=grid, cv=10, n_jobs=-1)
    xgbr_cv_s.fit(x_train, y_train)
    res_s = xgbr_cv_s.best_params_
    print(res_s)
    xgbr_s = XGBRegressor(n_estimators=res_s["n_estimators"], max_depth=res_s["max_depth"], eta=res_s["eta"], grow_policy=res_s["grow_policy"], eval_metric='rmse').fit(x_train, y_train)
    score = xgbr_s.score(x_test, y_test)
    preds = xgbr_s.predict(x_test)
    mse = mean_squared_error(y_test, preds)
    print("Test set R^2 Score: ", score)
    print("MSE: ", mse)
    print("RMSE: ", mse**(0.5))
    if score > 0.7:
        xgbr_s.save_model('private_gifts_model.json')
        input_frame.to_csv('private_gifts_per_student_per_dollar.csv')
        print("SAVED!")


In [754]:
def test_set(datapath, target, second = None, log = True, div_by_student = True, div_by_expense = True):
    df = pd.read_csv(datapath, low_memory=False)
    print(target)
    df_subset = df[['unitid', target]]
    df_subset.dropna(inplace=True, axis = 0)
    print(df_subset.shape)
    df_subset = df_subset.groupby('unitid').mean().reset_index()
    if second != None:
        df_subset[target] /= df[second]
        df_subset = df_subset.drop([second], axis=1)
    if div_by_student:
        df_subset = pd.merge(df_subset, total_students, how='inner', on='unitid')
        df_subset[target] /= df_subset['efytotlt']
        df_subset = df_subset.drop(['efytotlt'], axis = 1)
    if div_by_expense:
        expenses = fnce_data_final[['f2e131', 'unitid']]
        df_subset = pd.merge(df_subset, expenses, on='unitid', how='inner')
        print(df_subset.columns)
        df_subset[target] /= df_subset['f2e131']
        print(df_subset.shape)
        df_subset = df_subset.drop(['f2e131'], axis = 1)
    if log:
        df_subset[target] = np.log(df_subset[target])
        df_subset.replace([np.inf, -np.inf], np.nan, inplace=True)
        print(get_missing(df_subset))
        # df_subset = df_subset.fillna(df_subset.median())
        df_subset.dropna(inplace=True)
    merged = pd.merge(df_subset, fnce_data_final_features, how='inner', on='unitid')
    print(merged.shape)
    print(get_missing(merged))
    merged.dropna(inplace=True)
    
    X = merged.drop(['unitid', target], axis = 1)
    y = merged[target]
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    # y[np.array(np.where(y ==0))] = 1e-9
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    xgb_test(X, y, merged)
    # test(X, y)


In [758]:
test_set("datasets/Institutional Finances/F_F2_1415-1920_data.csv", 'f2d08', div_by_student=True, div_by_expense=True, log = True)

f2d08
(11198, 2)
Index(['unitid', 'f2d08', 'f2e131'], dtype='object')
(1840, 3)
        Percent Of Missing Values
unitid                       0.00
f2d08                       14.40
(1498, 16)
            Percent Of Missing Values
unitid                           0.00
f2d08                            0.00
f2b02                            0.00
f2c05                            0.00
f2c06                            0.00
f2d11                            0.00
f2d12                            0.00
f2e011                           0.00
f2e021                           0.00
f2e031                           0.00
f2e041                           0.00
f2e051                           0.00
f2e061                           0.00
f2e071                           0.00
f2e081                           0.00
expense_ps                       0.00
X shape: (1498, 14), y shape: (1498,)
{'n_estimators': 30, 'max_depth': 5, 'grow_policy': 'depthwise', 'eta': 0.15}
Test set R^2 Score:  0.7119178821400727
MSE: 